In [28]:
# stdlib
import syft as sy
from os import environ as env

In [29]:
# env["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# (environment := env.get("ORCHESTRA_DEPLOYMENT_TYPE", "python"))

In [10]:
# when running from tox these are set
ROOT_EMAIL = "admin@bigquery.org"
ROOT_PASSWORD = "bqpw"

In [13]:
# in case they were not set we set them now with the default value
# such that orchestra uses the same when launching the server
env["DEFAULT_ROOT_EMAIL"] = ROOT_EMAIL
env["DEFAULT_ROOT_PASSWORD"] = ROOT_PASSWORD

In [5]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    reset=True,
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [7]:
root_client = sy.login(
    url="http://localhost:8080", email=ROOT_EMAIL, password=ROOT_PASSWORD
)

Logged into <syft-dev-server: High side Datasite> as <admin@bigquery.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].account.set_password([new_password])`.

# Create new admin client

In [21]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"

In [22]:
# create
root_client.register(name="second admin", email=ADMIN_EMAIL, password=ADMIN_PW, password_verify=ADMIN_PW)
# update role
new_user_id = root_client.users.search(email=ADMIN_EMAIL)[0].id
root_client.users.update(uid=new_user_id, role="admin")

NameError: name 'user_email' is not defined

# We cannot delete the root client

In [24]:
high_client = sy.login(url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW)

Logged into <syft-dev-server: High side Datasite> as <admin2@bigquery.org>


In [25]:
root_admin_id = root_client.users.search(email=ROOT_EMAIL)[0].id

In [26]:
with sy.raises(sy.SyftException):
    high_client.users.delete(root_admin_id)

# Create ephemeral admin and delete it

In [10]:
# register 2nd new admin (to delete)
user_email, user_pw = "admin3@bigquery.org", "bqpw3"
## create
root_client.register(name="x", email=user_email, password=user_pw, password_verify=user_pw)
## update role
new_user_id2 = root_client.users.search(email=user_email)[0].id
root_client.users.update(uid=new_user_id, role="admin")

INFO:     127.0.0.1:49903 - "POST /api/v2/register HTTP/1.1" 200 OK
INFO:     127.0.0.1:49905 - "POST /api/v2/api_call HTTP/1.1" 200 OK
INFO:     127.0.0.1:49907 - "POST /api/v2/api_call HTTP/1.1" 200 OK


```python
class UserView:
  id: str = 7cf9d109e5a241beb9aae5340f63acbb
  name: str = "x"
  email: str = "admin2@bigquery.org"
  institution: str = None
  website: str = None
  role: str = ServiceRole.ADMIN
  notifications_enabled: str = {: True, : False, : False, : False}

```

In [11]:
root_client.users.delete(new_user_id2)

INFO:     127.0.0.1:49909 - "POST /api/v2/api_call HTTP/1.1" 200 OK


<UID: 5cef67d72d65400ba054e5b735add1a5>